In [ ]:
import sys, os

root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if root_path not in sys.path:
    sys.path.append(root_path)

In [ ]:
from src.data.load_cifrar100 import *
from src.data.load_cifrar100_C import *
import datasets
from datasets.utils.logging import disable_progress_bar

disable_progress_bar()

In [ ]:
pip -q install "datasets<=3.6.0" "huggingface_hub>=0.24"

In [ ]:
import aiohttp
from datasets import load_dataset

timeout = aiohttp.ClientTimeout(total=None, sock_read=2*60*60)

ds = load_dataset(
    "randall-lab/cifar100-c",
    split="test",
    trust_remote_code=True,
    storage_options={
        "client_kwargs": {"timeout": timeout},
        "block_size": 8 * 1024 * 1024})

## Test Cifrar100 clases 

In [5]:
from datasets.utils.logging import disable_progress_bar
disable_progress_bar()

verify_cifar100c_matches_torchvision(ds, data_dir="./data", corruption="gaussian_noise")

HF len: 10000 TV len: 10000
HF label min/max: 0 99 unique: 100
TV label min/max: 0 99 unique: 100
Label match rate (HF vs torchvision): 100.00%


In [6]:
verify_cifar100c_matches_torchvision(ds, data_dir="./data", corruption="motion_blur")

HF len: 10000 TV len: 10000
HF label min/max: 0 99 unique: 100
TV label min/max: 0 99 unique: 100
Label match rate (HF vs torchvision): 100.00%


In [7]:
verify_cifar100c_matches_torchvision(ds, data_dir="./data", corruption="jpeg_compression")

HF len: 10000 TV len: 10000
HF label min/max: 0 99 unique: 100
TV label min/max: 0 99 unique: 100
Label match rate (HF vs torchvision): 100.00%


In [8]:
verify_cifar100c_matches_torchvision(ds, data_dir="./data", corruption="fog")

HF len: 10000 TV len: 10000
HF label min/max: 0 99 unique: 100
TV label min/max: 0 99 unique: 100
Label match rate (HF vs torchvision): 100.00%


In [9]:
verify_cifar100c_matches_torchvision(ds, data_dir="./data", corruption="motion_blur")

HF len: 10000 TV len: 10000
HF label min/max: 0 99 unique: 100
TV label min/max: 0 99 unique: 100
Label match rate (HF vs torchvision): 100.00%


In [10]:
train_loader, val_loader, test_loader = get_cifar100_dataloaders(
    batch_size=64,
    data_dir="./data",
    num_workers=2,
    val_split=0.1,
    img_size=32 , seed=7)


---


In [ ]:
from src.Model_A_OutGridNet import * 
from src.stage_config import *
from src.training.chekpoints import * 
from src.training.one_epoch_train import *

def Abliations_no_outlooker(drop_path=0.08):
    return [
        StageCfg(dim=64,  depth=2, num_heads=2, grid_size=8, outlook_heads=0, drop_path=0.0),
        StageCfg(dim=128, depth=2, num_heads=4, grid_size=8, outlook_heads=0, drop_path=0.0),
        StageCfg(dim=256, depth=3, num_heads=8, grid_size=4, outlook_heads=0, drop_path=0.0),
        StageCfg(dim=384, depth=1, num_heads=6, grid_size=2, outlook_heads=0, drop_path=0.0),]

stages = Abliations_no_outlooker()

model1 = MaxOutNet(
    num_classes=100,
    stages=stages,
    stem_dim=64,
    dpr_max=0.05)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
def Abliations_baseline(drop_path=0.08):
    return [
        StageCfg(dim=64,  depth=2, num_heads=2, grid_size=8, outlook_heads=2, drop_path=0.08),
        StageCfg(dim=128, depth=2, num_heads=4, grid_size=8, outlook_heads=4, drop_path=0.08),
        StageCfg(dim=256, depth=3, num_heads=8, grid_size=4, outlook_heads=8, drop_path=0.08),
        StageCfg(dim=384, depth=1, num_heads=6, grid_size=2, outlook_heads=6, drop_path=0.08),]

stages = Abliations_baseline()

model = MaxOutNet(
    num_classes=100,
    stages=stages,
    stem_dim=64,
    dpr_max=0.1)

In [ ]:
def Abliations_no_grid(drop_path=0.08):
    return [
        StageCfg(dim=64,  depth=2, num_heads=0, grid_size=8, outlook_heads=2, drop_path=0.0),
        StageCfg(dim=128, depth=2, num_heads=0, grid_size=8, outlook_heads=4, drop_path=0.0),
        StageCfg(dim=256, depth=3, num_heads=0, grid_size=4, outlook_heads=8, drop_path=0.0),
        StageCfg(dim=384, depth=1, num_heads=0, grid_size=2, outlook_heads=6, drop_path=0.0),]

stages = Abliations_no_grid()

model2 = MaxOutNet(
    num_classes=100,
    stages=stages,
    stem_dim=64,
    dpr_max=0.1)

In [ ]:
def count_trainable_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

n_params = count_trainable_parameters(model)
print(f"Trainable parameters: {n_params:,}")

Trainable parameters: 12,187,300


In [ ]:
load_checkpoint(path='/content/last_No_Outlooker_cifrar32.pt' ,model=model1)

In [ ]:
load_checkpoint(path='/content/last_Hybrid_Full_Cifrar32.pt' ,model=model)

In [ ]:
load_checkpoint(path='/content/last_No_Grid_cifrar32.pt' ,model=model2)

---

In [ ]:
evaluate_one_epoch(model=model,dataloader=test_loader)

(0.828532830619812, {'top1': 79.8, 'top3': 91.73, 'top5': 94.91})

In [ ]:
evaluate_one_epoch(model=model1, dataloader=test_loader)


(0.7754235710144043, {'top1': 79.43, 'top3': 91.59, 'top5': 94.77})

In [ ]:
evaluate_one_epoch(model=model2, dataloader=test_loader)

(0.749603179359436, {'top1': 80.64, 'top3': 92.03, 'top5': 94.92})

---

## Full Model Cifrar100

In [ ]:
from datasets.utils.logging import disable_progress_bar
disable_progress_bar()

quick_corruptions = ["gaussian_noise", "motion_blur", "jpeg_compression", "fog", "pixelate"]

results_quick = evaluate_tinyc_suite(
    model=model,
    base_ds=ds,
    evaluate_one_epoch_fn=evaluate_one_epoch,
    reference_clean_loader=test_loader,
    batch_size=128,
    img_size=64,
    num_workers=2,
    corruptions=quick_corruptions,
    severities=(1, 3, 5),
    verbose=True)

print(summarize_tinyc_results(results_quick, metric_key="top1"))
print(summarize_tinyc_results(results_quick, metric_key="top5"))

[Clean] loss 0.8285 | top1 79.80% | top3 91.73% | top5 94.91%
[  gaussian_noise | s=1] loss 5.0415 | top1 6.23% | top3 15.75% | top5 21.43%
[  gaussian_noise | s=3] loss 5.4826 | top1 3.81% | top3 10.27% | top5 14.02%
[  gaussian_noise | s=5] loss 5.5669 | top1 3.35% | top3 9.10% | top5 12.32%
[     motion_blur | s=1] loss 4.4916 | top1 8.63% | top3 19.06% | top5 27.79%
[     motion_blur | s=3] loss 5.0884 | top1 4.41% | top3 12.45% | top5 20.05%
[     motion_blur | s=5] loss 5.3036 | top1 3.73% | top3 10.65% | top5 18.39%
[jpeg_compression | s=1] loss 4.4895 | top1 7.89% | top3 19.35% | top5 26.81%
[jpeg_compression | s=3] loss 4.6923 | top1 6.07% | top3 14.83% | top5 21.38%
[jpeg_compression | s=5] loss 4.8028 | top1 5.52% | top3 13.22% | top5 18.93%
[             fog | s=1] loss 3.9933 | top1 13.50% | top3 27.62% | top5 37.63%
[             fog | s=3] loss 4.5291 | top1 8.31% | top3 20.25% | top5 29.56%
[             fog | s=5] loss 5.4004 | top1 3.05% | top3 13.48% | top5 21.70%
[ 

## No Grid

In [ ]:
results_quick = evaluate_tinyc_suite(
    model=model2,
    base_ds=ds,
    evaluate_one_epoch_fn=evaluate_one_epoch,
    reference_clean_loader=test_loader,
    batch_size=128,
    img_size=64,
    num_workers=2,
    corruptions=quick_corruptions,
    severities=(1, 3, 5),
    verbose=True)

print(summarize_tinyc_results(results_quick, metric_key="top1"))
print(summarize_tinyc_results(results_quick, metric_key="top5"))

[Clean] loss 0.7520 | top1 80.53% | top3 92.07% | top5 94.82%
[  gaussian_noise | s=1] loss 5.3241 | top1 5.85% | top3 14.63% | top5 21.33%
[  gaussian_noise | s=3] loss 5.9538 | top1 3.13% | top3 8.55% | top5 14.24%
[  gaussian_noise | s=5] loss 6.1656 | top1 2.32% | top3 7.39% | top5 12.11%
[     motion_blur | s=1] loss 5.1857 | top1 6.75% | top3 18.16% | top5 26.95%
[     motion_blur | s=3] loss 5.9643 | top1 3.47% | top3 11.36% | top5 18.43%
[     motion_blur | s=5] loss 6.2144 | top1 2.80% | top3 10.15% | top5 16.67%
[jpeg_compression | s=1] loss 5.2256 | top1 5.71% | top3 19.08% | top5 28.33%
[jpeg_compression | s=3] loss 5.5295 | top1 4.07% | top3 13.90% | top5 21.98%
[jpeg_compression | s=5] loss 5.6463 | top1 3.70% | top3 11.92% | top5 18.50%
[             fog | s=1] loss 4.3974 | top1 12.54% | top3 27.97% | top5 38.57%
[             fog | s=3] loss 5.1985 | top1 7.14% | top3 19.61% | top5 29.60%
[             fog | s=5] loss 6.4141 | top1 2.15% | top3 10.68% | top5 19.13%
[  

In [ ]:
results_quick = evaluate_tinyc_suite(
    model=model1,
    base_ds=ds,
    evaluate_one_epoch_fn=evaluate_one_epoch,
    reference_clean_loader=test_loader,
    batch_size=128,
    img_size=64,
    num_workers=2,
    corruptions=quick_corruptions,
    severities=(1, 3, 5),
    verbose=True)

print(summarize_tinyc_results(results_quick, metric_key="top1"))
print(summarize_tinyc_results(results_quick, metric_key="top5"))

[Clean] loss 0.7754 | top1 79.43% | top3 91.59% | top5 94.77%
[  gaussian_noise | s=1] loss 5.5587 | top1 6.47% | top3 16.75% | top5 22.73%
[  gaussian_noise | s=3] loss 6.1951 | top1 3.36% | top3 10.46% | top5 14.22%
[  gaussian_noise | s=5] loss 6.2861 | top1 2.89% | top3 8.90% | top5 12.59%
[     motion_blur | s=1] loss 5.3823 | top1 7.49% | top3 17.70% | top5 25.80%
[     motion_blur | s=3] loss 6.2469 | top1 3.76% | top3 10.61% | top5 17.94%
[     motion_blur | s=5] loss 6.5048 | top1 3.27% | top3 9.30% | top5 16.42%
[jpeg_compression | s=1] loss 5.2316 | top1 7.09% | top3 17.93% | top5 25.83%
[jpeg_compression | s=3] loss 5.5491 | top1 5.36% | top3 13.51% | top5 20.07%
[jpeg_compression | s=5] loss 5.6929 | top1 4.75% | top3 12.00% | top5 17.76%
[             fog | s=1] loss 4.4647 | top1 13.59% | top3 28.40% | top5 37.76%
[             fog | s=3] loss 5.2869 | top1 7.90% | top3 20.75% | top5 30.53%
[             fog | s=5] loss 6.5490 | top1 2.45% | top3 13.60% | top5 20.93%
[  

---

## For reference train Resnet18

In [25]:
import timm
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"

model_resnet = timm.create_model("resnet18", pretrained=False, num_classes=100).to(device)

model_resnet.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
model_resnet.maxpool = nn.Identity()

ckpt = load_checkpoint(
    "/content/last_model_resnet18.pt",
    model_resnet,
    map_location=device,
    strict=True)

In [26]:
quick_corruptions = ["gaussian_noise", "motion_blur", "jpeg_compression", "fog", "pixelate"]

results_quick = evaluate_tinyc_suite(
    model=model_resnet,
    base_ds=ds,
    evaluate_one_epoch_fn=evaluate_one_epoch,
    reference_clean_loader=test_loader,
    batch_size=128,
    img_size=64,
    num_workers=2,
    corruptions=quick_corruptions,
    severities=(1, 3, 5),
    verbose=True)

print(summarize_tinyc_results(results_quick, metric_key="top1"))
print(summarize_tinyc_results(results_quick, metric_key="top5"))

[Clean] loss 1.0124 | top1 73.28% | top3 88.26% | top5 91.99%
[  gaussian_noise | s=1] loss 5.0139 | top1 4.13% | top3 13.15% | top5 17.77%
[  gaussian_noise | s=3] loss 5.2251 | top1 2.32% | top3 8.05% | top5 11.97%
[  gaussian_noise | s=5] loss 5.2581 | top1 2.06% | top3 6.77% | top5 10.18%
[     motion_blur | s=1] loss 5.1240 | top1 4.49% | top3 13.49% | top5 19.58%
[     motion_blur | s=3] loss 5.6696 | top1 2.44% | top3 9.63% | top5 15.55%
[     motion_blur | s=5] loss 5.8629 | top1 2.15% | top3 8.55% | top5 14.29%
[jpeg_compression | s=1] loss 4.9442 | top1 4.74% | top3 13.78% | top5 19.58%
[jpeg_compression | s=3] loss 5.1476 | top1 3.87% | top3 11.62% | top5 17.00%
[jpeg_compression | s=5] loss 5.2490 | top1 3.50% | top3 10.70% | top5 15.81%
[             fog | s=1] loss 4.6183 | top1 7.31% | top3 17.76% | top5 24.51%
[             fog | s=3] loss 4.9450 | top1 4.76% | top3 15.29% | top5 21.96%
[             fog | s=5] loss 5.6819 | top1 1.99% | top3 10.22% | top5 16.74%
[     

---

## Pretrained ResNet18 for more reference 

In [ ]:
!pip install detectors

In [ ]:
import detectors

device = "cuda" if torch.cuda.is_available() else "cpu"
model = timm.create_model("resnet18_cifar100", pretrained=True).to(device)
model.eval()

In [30]:
quick_corruptions = ["gaussian_noise", "motion_blur", "jpeg_compression", "fog", "pixelate"]

results_quick = evaluate_tinyc_suite(
    model=model,
    base_ds=ds,
    evaluate_one_epoch_fn=evaluate_one_epoch,
    reference_clean_loader=test_loader,
    batch_size=128,
    img_size=64,
    num_workers=2,
    corruptions=quick_corruptions,
    severities=(1, 3, 5),
    verbose=True)

print(summarize_tinyc_results(results_quick, metric_key="top1"))
print(summarize_tinyc_results(results_quick, metric_key="top5"))

[Clean] loss 0.8644 | top1 79.27% | top3 91.65% | top5 94.53%
[  gaussian_noise | s=1] loss 4.6187 | top1 5.48% | top3 12.19% | top5 17.30%
[  gaussian_noise | s=3] loss 4.9510 | top1 2.56% | top3 7.68% | top5 11.01%
[  gaussian_noise | s=5] loss 5.0964 | top1 2.20% | top3 6.15% | top5 9.21%
[     motion_blur | s=1] loss 5.2906 | top1 4.98% | top3 11.69% | top5 19.00%
[     motion_blur | s=3] loss 6.4385 | top1 2.39% | top3 5.98% | top5 12.34%
[     motion_blur | s=5] loss 6.9049 | top1 2.21% | top3 5.02% | top5 10.47%
[jpeg_compression | s=1] loss 5.0930 | top1 4.82% | top3 12.51% | top5 19.41%
[jpeg_compression | s=3] loss 5.0411 | top1 3.69% | top3 10.11% | top5 15.81%
[jpeg_compression | s=5] loss 4.9698 | top1 3.40% | top3 8.95% | top5 14.17%
[             fog | s=1] loss 4.7815 | top1 8.23% | top3 18.09% | top5 26.30%
[             fog | s=3] loss 5.8132 | top1 4.32% | top3 12.00% | top5 18.63%
[             fog | s=5] loss 7.5449 | top1 2.01% | top3 6.79% | top5 11.97%
[        